<a href="https://colab.research.google.com/github/VikramAchuthan/HockeyDataset/blob/master/FinishedHockeyDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import initial libraries

In [11]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
import pandas as pd
import numpy as np

Retrieve data of all D1 and D2 NCAA women's hockey colleges from separate google sheet spreadsheet


In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Th01ggWcHbjQGE6HBYstuiGHbO4JTcq8FnbKgL5PzXc/edit#gid=0')

collegesSheet = wb.worksheet('NCAA D1 and D2 Schools')

collegesData = collegesSheet.get_all_values()
print(collegesData)

Store colleges in panda dataframe


In [ ]:

collegeDF = pd.DataFrame(collegesData)
collegeDF.columns = collegeDF.iloc[0]
collegeDF = collegeDF.iloc[1:]

collegeDF.head(40)
print(collegeDF.columns)

Scrape data of all players who committed to schools to play Women's Ice Hockey from collegecommitments.com.

*Note: Collegecommitments.com does not host requisite data currently, so this is not required. *`

In [ ]:



import requests
import lxml.html as lh
#website with data
url='https://collegecommitments.com/CommitList.aspx?x7cq9=FEMALE'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')


tr_elements_new = tr_elements[5:]



In [ ]:
col=[]
i=0
tableHead = ['Date','Player Name', 'Pos.','College','Current Team', 'Starting Year']

#For each row, store each first element (header) and an empty list
for x in range(0,6):
    i+=1
    name=tableHead[x]
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

for j in range(1,len(tr_elements_new)):
    #T is our j'th row
    T=tr_elements_new[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=6:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

#create panda dataframe of all players who committed 
playerDataDict={title:column for (title,column) in col}
playerDataDF=pd.DataFrame(playerDataDict)
playerDataDF.head(950)


Scrape prerecorded data from google spreadsheet

In [ ]:
playerSheet = wb.worksheet('Players Information')

playerData = playerSheet.get_all_values()

playerDataDF = pd.DataFrame(playerData)
playerDataDF.columns = playerDataDF.iloc[0]
playerDataDF = playerDataDF.iloc[1:]

playerDataDF.head(950)



Filter to include only Players who committed to D1 and D2 schools and create data frame of those players

In [ ]:

print(len(playerDataDF))
filteredPlayers = []
for x in range(len(playerDataDF)):
  committedCollege =  playerDataDF.iloc[x][3]
  nameList = playerDataDF.iloc[x][1].split(",")
  playerDataDF.iloc[x][1] = nameList[1] + " " + nameList[0].title()
  if collegeDF['College'].str.contains(committedCollege.strip()).any():
    if(playerDataDF.iloc[x][1] == 'Lydia Murray'): 
      print("YES!!!!!")
    filteredPlayers.append([playerDataDF.iloc[x][0],playerDataDF.iloc[x][1],playerDataDF.iloc[x][2],playerDataDF.iloc[x][3],playerDataDF.iloc[x][4],playerDataDF.iloc[x][5]])
print(filteredPlayers)
filteredPlayersDF = pd.DataFrame(filteredPlayers)
filteredPlayersDF.columns = playerDataDF.columns
filteredPlayersDF.head(600)


Create a dictionary with a key of D1 and D2 WIH colleges, and their corresponding statistics webpage

In [ ]:
#add to google spreadsheet
# worksheet = wb.add_worksheet(title="Filtered Hockey Players", rows="1000", cols="20")
# worksheet.update([filteredPlayersDF.columns.values.tolist()] + filteredPlayersDF.values.tolist())

#create a dictionary matching college to hockey data for that college
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Th01ggWcHbjQGE6HBYstuiGHbO4JTcq8FnbKgL5PzXc/edit#gid=0')

collegesSheet = wb.worksheet('College Statistics Websites')

collegeStatWebsiteList = collegesSheet.get_all_values()
collegeStatWebsiteDict = {}
for college in collegeStatWebsiteList:
  collegeStatWebsiteDict[college[0]] = college[1]
print(collegeStatWebsiteDict)






*  Ask user to enter class (freshman,sophomore...)
*  Create a new dataset based on class of college WIH players to compare their statistics. 

For example, if the user entered "1" - corresponding to the sophomore class - the stats from all sophomores from 2014 to 2019 will be scraped and stored in a single dataset. The data is organized in this way to allow for a fair comparison and analysis. 


In [ ]:

from datetime import datetime

yearToCodeDict = {'2014':'1415','2015':'1516','2016':'1617','2017':'1718','2018':'1819','2019':'1920','2020':'2021'}

experience = int(input("Enter the experience: 0,1,2,3: "))

count=0
playerProfileDict = {}
for season in range(2014,2019):
  for x in range(len(filteredPlayersDF)):
    startingYear = int(filteredPlayersDF.iloc[x][5][:4]) #entry year
    if (startingYear == season-experience) and filteredPlayersDF.iloc[x][3] in collegeStatWebsiteDict.keys() and filteredPlayersDF.iloc[x][2] != 'G':
      playerName = filteredPlayersDF.iloc[x][1] 
      #print("Player Name: {}    Starting Year: {}      Season: {}".format(playerName,startingYear,season))
      college = filteredPlayersDF.iloc[x][3]
      collegeWebsite = collegeStatWebsiteDict[college]
      collegeWebsite = collegeWebsite.replace('1819', yearToCodeDict[str(season)])
    #  print("Website",collegeWebsite)
    # print("website: {}".format(collegeWebsite))
      statisticForSeasonDF = pd.read_html(collegeWebsite)
    # print("COLLEGE: {}      {}".format(college,playerName))
      statisticForSeasonDF = pd.DataFrame(statisticForSeasonDF[1])
      selectedPlayerStatistics = statisticForSeasonDF.loc[statisticForSeasonDF[1] == playerName.strip()].to_string()
  
      indexofname = selectedPlayerStatistics.find(playerName)
      selectedPlayerStatistics = selectedPlayerStatistics[indexofname:]
      #itemize full list into individual stats
      selectedPlayerStatistics = selectedPlayerStatistics.split()
      selectedPlayerStatistics = selectedPlayerStatistics[2:]
      #convert dates (string object) to datetime objects
      committed_date_time_str = filteredPlayersDF.iloc[x][0]
      #print(committed_date_time_str)
      committed_date_time_obj = datetime.strptime(committed_date_time_str, '%m/%d/%y') 
      entry_date_time_str = '09/01/'+filteredPlayersDF.iloc[x][5][2:4]
      entry_date_time_obj = datetime.strptime(entry_date_time_str, '%m/%d/%y') 
      difference = entry_date_time_obj-committed_date_time_obj
      elapsedTimeMonths = (difference.total_seconds())/2592000
      playerProfileList = [playerName,college,entry_date_time_obj,committed_date_time_obj,round(elapsedTimeMonths,2),season]

      x = 0
      integerIndexes = [2,3,4,5,7,8,9,11,12,13,14,15,17,18,19,21,22,23,24,25]
      for statistic in selectedPlayerStatistics:
        if x in integerIndexes and statistic !='NaN': 
          statistic = int(statistic)
        x +=1
        playerProfileList.append(statistic)
      #print(len(playerProfileList))
      if len(playerProfileList) < 31:
      # print("YEYEYEYEY",playerName,len(playerProfileList))
        count+=1
        continue
    # playerProfileList.append(filteredPlayersDF.iloc[x][5]) #appending year committed in string format  for convenience
      playerProfileDict[playerName] = playerProfileList

    
  



playerProfileDF = pd.DataFrame.from_dict(playerProfileDict, orient = 'index')
playerProfileDF.columns = ['Player Name','College','Entry Year','Date Committed', 'Elapsed Months','Statistics From','Position',	'YR	','Overall GP',	'Overall G'	,'Overall A',	'Overall PTS'	,'Overall PEN/MIN',	'Overall PP',	'Overall SH'	,
                          'Overall GW'	,'Overall +/-',	'Overall SOG',	'Conference GP',	'Conference G',	'Conference A',	'Conference PTS',	'Conference PEN/MIN',	'Conference PP',	'Conference SH',	'Conference GW'	,'Conference +/-',	'Conference SOG',	'Career GP',	'Career G',	'Career A',	'Career PT']
#playerProfileDF[playerProfileDF.POS.str.contains(None) == False]
playerProfileDF.head(700)


Connect Google Drive to Google Colab

In [ ]:
from google.colab import drive
drive.mount('/drive')


Analyze the data using [Plotly](https://plotly.com/python/), the Python Graphing Library

The user will be asked to input a "Y" variable to be compared to the "X" variable - elapsed time between commit date and entry date. 

In [ ]:



import matplotlib.pyplot as plt
import plotly.express as px
yVariable = str(input("Enter the statistic you wish to measure against elapsed time:"))
graphedDataList = []

for key in playerProfileDF['Elapsed Months'].keys(): #key is the player name
  graphedDataList.append([key,playerProfileDF['Position'][key],playerProfileDF['Elapsed Months'][key],playerProfileDF[yVariable][key],playerProfileDF['College'][key],playerProfileDF['Statistics From'][key],playerProfileDF['Overall GP'][key]])

sumX = 0
sumY = 0
for each in graphedDataList:
  sumX = sumX + float(each[2])
  sumY = sumY + int(each[3])
graphedDataList.append(['MEAN','MEAN',sumX/len(graphedDataList),sumY/len(graphedDataList),'NA','NA','NA'])
graphedDataDF = pd.DataFrame(graphedDataList)
graphedDataDF.columns = ['Player Name','Position', 'Elapsed Months', yVariable,'College','Statistics From','Overall GP']
graphedDataDF.head(60)



experienceToClassDict = {0:'Freshmen',1:"Sophomore",2:"Junior",3:'Senior'}


fig = px.scatter(graphedDataDF, x="Elapsed Months", y=yVariable,color = 'Position',hover_name="Player Name",hover_data=["College",'Statistics From','Overall GP'])#hover_data=["continent", "pop"])
#fig.update_traces(marker_color='blue',line_color='red')
fig.update_layout(
    title= yVariable + " vs Elapsed Months, "+experienceToClassDict[experience] + " Hockey Players" ,
 
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()
fig.write_html("/drive/My Drive/"+experienceToClassDict[experience]+"HockeyDataset.html")



